In [1]:
import os
import numpy as np
from keras.preprocessing import image
from keras.applications import VGG16, Xception, InceptionV3, ResNet50, EfficientNetV2L
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tqdm import tqdm  # Para barra de progresso

# Settings
root_directory = r'MED115\(sub)_MED_115_database_classes'
chosen_model = 'efficientnetv2l'  # ou 'xception', 'inceptionv3', 'resnet50', 'efficientnetv2l'

# Escolhe modelo
if chosen_model == 'vgg16':
    base_model = VGG16(weights='imagenet', include_top=False)
    input_size = (224, 224)
elif chosen_model == 'xception':
    base_model = Xception(weights='imagenet', include_top=False)
    input_size = (299, 299)
elif chosen_model == 'inceptionv3':
    base_model = InceptionV3(weights='imagenet', include_top=False)
    input_size = (299, 299)
elif chosen_model == 'resnet50':
    base_model = ResNet50(weights='imagenet', include_top=False)
    input_size = (224, 224)
elif chosen_model == 'efficientnetv2l':
    base_model = EfficientNetV2L(weights='imagenet', include_top=False)
    input_size = (224, 224)
else:
    raise ValueError("Modelo desconhecido selecionado.")

X = []
y = []

# Coleta caminhos de imagens primeiro para usar com tqdm
all_image_paths = []
for class_name in os.listdir(root_directory):
    class_dir = os.path.join(root_directory, class_name)
    if os.path.isdir(class_dir):
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                all_image_paths.append((class_name, os.path.join(class_dir, filename)))

# Processa imagens com barra de progresso
for class_name, img_path in tqdm(all_image_paths, desc="Processando imagens"):
    try:
        img = image.load_img(img_path, target_size=input_size, color_mode='rgb')
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0

        features = base_model.predict(img, verbose=0).flatten()
        X.append(features)
        y.append(class_name)
    except Exception as e:
        print(f"Erro ao carregar {img_path}: {e}")

X = np.array(X)
y = np.array(y)

# Divide em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Treina RandomForest
print("Treinando Random Forest...")
clf = RandomForestClassifier(n_estimators=500, random_state=42)
clf.fit(X_train, y_train)

# Avaliação
print("Avaliando modelo...")
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) * 100
f1 = f1_score(y_test, y_pred, average='weighted') * 100
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0) * 100
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0) * 100

print(f"\nModelo: {chosen_model}")
print(f"Acurácia: {accuracy:.2f}%")
print(f"F1 Score: {f1:.2f}%")
print(f"Precisão: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")

Processando imagens: 100%|██████████████████████████████████████████████████████| 8809/8809 [11:53:58<00:00,  4.86s/it]


Treinando Random Forest...
Avaliando modelo...

Modelo: efficientnetv2l
Acurácia: 87.59%
F1 Score: 86.90%
Precisão: 88.85%
Recall: 87.59%
